In [21]:
import pandas as pd
import numpy as np
import gensim
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, log_loss
from sklearn import preprocessing
import re
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
import xgboost as xgb

from tqdm import tqdm

nltk.download('stopwords')
nltk.download('punkt')
stops = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/home/ubuntu/Envs/nlp/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [7]:
train = pd.read_csv('train.csv')
print(len(train))
train.head()

19579


,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [6]:
test = pd.read_csv('test.csv')
print(len(test))
test.head()

8392


,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [8]:
label_enconder = preprocessing.LabelEncoder()
label_enconder.fit(train['author'])
train['label_encoded'] = label_enconder.transform(train['author'])
train.head()

,id,text,author,label_encoded
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1


## Preprocessing function

In [178]:
def transformText(text):
    
    stops = set(stopwords.words("english"))
    
    # Convert text to lower
    text = text.lower()
    # Removing non ASCII chars    
    text = re.sub(r'[^\x00-\x7f]',r' ',text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Removing all the stopwords
    filtered_words = [word for word in text.split() if word not in stops]
    #filtered_words = [word for word in text.split()]

    # Removing all the tokens with lesser than 3 characters
    filtered_words = gensim.corpora.textcorpus.remove_short(filtered_words, minsize=3)
    
    # Preprocessed text after stop words removal
    text = " ".join(filtered_words)
    
    # Remove the punctuation
    text = gensim.parsing.preprocessing.strip_punctuation2(text)
    
    # Strip all the numerics
    text = gensim.parsing.preprocessing.strip_numeric(text)
    
    # Strip multiple whitespaces
    text = gensim.corpora.textcorpus.strip_multiple_whitespaces(text)
    
    # Stemming
    text = gensim.parsing.preprocessing.stem_text(text)
    return text

## Creating preprocessing column

In [180]:
train['text_processed']=train['text'].apply(lambda x: transformText(x))
train.tail()

,id,text,author,label_encoded,text_processed
19574,id17718,"I could have fancied, while I looked at it, th...",EAP,0,could fanci look it emin landscap painter buil...
19575,id08973,The lids clenched themselves together as if in...,EAP,0,lid clench togeth spasm
19576,id05267,"Mais il faut agir that is to say, a Frenchman ...",EAP,0,mai faut agir sai frenchman never faint outright
19577,id17513,"For an item of news like this, it strikes us i...",EAP,0,item new like thi strike coolli receiv
19578,id00393,"He laid a gnarled claw on my shoulder, and it ...",HPL,1,laid gnarl claw shoulder seem shake altogeth m...


In [181]:
test['text_processed']=test['text'].apply(lambda x: transformText(x))
print(len(test))
test.tail()

8392


,id,text,text_processed
8387,id11749,All this is now the fitter for my purpose.,fitter purpos
8388,id10526,I fixed myself on a wide solitude.,fix wide solitud
8389,id13477,It is easily understood that what might improv...,easili understood might improv close scrutin d...
8390,id13761,"Be this as it may, I now began to feel the ins...",mai began feel inspir burn hope length nurtur ...
8391,id04282,"Long winded, statistical, and drearily genealo...",long wind statist drearili genealog matter wa ...


## Train/Test split

In [182]:
x_train, x_test, y_train, y_test = train_test_split(train['text_processed'], train['label_encoded'], test_size = 0.2, random_state = 4)
true_label = np.array(y_test)
print("#" * 20 + " Some stats " + "#"*20)
print("Dataset training: {} uterances".format(x_train.shape[0]))
print("Dataset testing: {} uterances".format(x_test.shape[0]))
print("Different classes: {}".format(len(y_train.unique())))

#################### Some stats ####################
Dataset training: 15663 uterances
Dataset testing: 3916 uterances
Different classes: 3


## Generating features

In [42]:
# TFIDF features
tfv = TfidfVectorizer(min_df=3,  max_features=None, ngram_range=(1, 2), use_idf=1,smooth_idf=1,sublinear_tf=1)
tfv.fit(list(x_train) + list(x_test))
x_train_tfv =  tfv.transform(x_train) 
x_test_tfv = tfv.transform(x_test)

In [183]:
# Count features
cnt = CountVectorizer(ngram_range=(1, 2))
cnt.fit(list(x_train) + list(x_test))
x_train_cnt =  cnt.transform(x_train) 
x_test_cnt = cnt.transform(x_test)

In [184]:
x_test_cnt.shape

(3916, 224257)

## Training XGBoost Classifer w/ early stopping on TFIDF features

In [174]:
clf = xgb.XGBClassifier(max_depth=4, n_estimators=3000, colsample_bytree=0.8, 
                        subsample=0.5, nthread=10, learning_rate=0.1)
eval_set = [(x_test_tfv.tocsc(),y_test)]
clf.fit(x_train_tfv.tocsc(), y_train, eval_metric = "mlogloss", eval_set=eval_set, verbose=True)
preds_proba = clf.predict_proba(x_test_tfv.tocsc())
preds = clf.predict(x_test_tfv.tocsc())

[0]	validation_0-mlogloss:1.08754
[1]	validation_0-mlogloss:1.07767
[2]	validation_0-mlogloss:1.06903
[3]	validation_0-mlogloss:1.06118
[4]	validation_0-mlogloss:1.05449
[5]	validation_0-mlogloss:1.04843
[6]	validation_0-mlogloss:1.0428
[7]	validation_0-mlogloss:1.03744
[8]	validation_0-mlogloss:1.03219
[9]	validation_0-mlogloss:1.02764
[10]	validation_0-mlogloss:1.02293
[11]	validation_0-mlogloss:1.01903
[12]	validation_0-mlogloss:1.01477
[13]	validation_0-mlogloss:1.01083
[14]	validation_0-mlogloss:1.00728
[15]	validation_0-mlogloss:1.00399
[16]	validation_0-mlogloss:1.00041
[17]	validation_0-mlogloss:0.997004
[18]	validation_0-mlogloss:0.99369
[19]	validation_0-mlogloss:0.990299
[20]	validation_0-mlogloss:0.987696
[21]	validation_0-mlogloss:0.985076
[22]	validation_0-mlogloss:0.982063
[23]	validation_0-mlogloss:0.979313
[24]	validation_0-mlogloss:0.976583
[25]	validation_0-mlogloss:0.97397
[26]	validation_0-mlogloss:0.971155
[27]	validation_0-mlogloss:0.968525
[28]	validation_0-mlog

KeyboardInterrupt: 

## Training XGBoost Classifer w/ early stopping on count features

In [ ]:
clf = xgb.XGBClassifier(max_depth=5, n_estimators=3000, colsample_bytree=0.8, 
                        subsample=0.5, nthread=10, learning_rate=0.1)
eval_set = [(x_test_cnt.tocsc(),y_test)]
clf.fit(x_train_cnt.tocsc(), y_train, eval_metric = "mlogloss", eval_set=eval_set, verbose=True)
#clf.fit(x_train_cnt.tocsc(), y_train)
preds_proba = clf.predict_proba(x_test_cnt.tocsc())
preds = clf.predict(x_test_cnt.tocsc())

[0]	validation_0-mlogloss:1.08762
[1]	validation_0-mlogloss:1.07743
[2]	validation_0-mlogloss:1.06763
[3]	validation_0-mlogloss:1.05862
[4]	validation_0-mlogloss:1.05107
[5]	validation_0-mlogloss:1.04399
[6]	validation_0-mlogloss:1.03705
[7]	validation_0-mlogloss:1.03069
[8]	validation_0-mlogloss:1.02466
[9]	validation_0-mlogloss:1.0192
[10]	validation_0-mlogloss:1.01402
[11]	validation_0-mlogloss:1.00887
[12]	validation_0-mlogloss:1.00422
[13]	validation_0-mlogloss:1.00024
[14]	validation_0-mlogloss:0.996283
[15]	validation_0-mlogloss:0.991872
[16]	validation_0-mlogloss:0.988181
[17]	validation_0-mlogloss:0.984671
[18]	validation_0-mlogloss:0.98093
[19]	validation_0-mlogloss:0.977677
[20]	validation_0-mlogloss:0.974193
[21]	validation_0-mlogloss:0.970882
[22]	validation_0-mlogloss:0.967865
[23]	validation_0-mlogloss:0.96494
[24]	validation_0-mlogloss:0.961749
[25]	validation_0-mlogloss:0.958871
[26]	validation_0-mlogloss:0.956115
[27]	validation_0-mlogloss:0.953397
[28]	validation_0-m

[226]	validation_0-mlogloss:0.735611
[227]	validation_0-mlogloss:0.735034
[228]	validation_0-mlogloss:0.734442
[229]	validation_0-mlogloss:0.733832
[230]	validation_0-mlogloss:0.733273
[231]	validation_0-mlogloss:0.732662
[232]	validation_0-mlogloss:0.732071
[233]	validation_0-mlogloss:0.73168
[234]	validation_0-mlogloss:0.731015
[235]	validation_0-mlogloss:0.730355
[236]	validation_0-mlogloss:0.729641
[237]	validation_0-mlogloss:0.729066
[238]	validation_0-mlogloss:0.728765
[239]	validation_0-mlogloss:0.728241
[240]	validation_0-mlogloss:0.727737
[241]	validation_0-mlogloss:0.72737
[242]	validation_0-mlogloss:0.726975
[243]	validation_0-mlogloss:0.726398
[244]	validation_0-mlogloss:0.725912
[245]	validation_0-mlogloss:0.72532
[246]	validation_0-mlogloss:0.724961
[247]	validation_0-mlogloss:0.724483
[248]	validation_0-mlogloss:0.724002
[249]	validation_0-mlogloss:0.723642
[250]	validation_0-mlogloss:0.722923
[251]	validation_0-mlogloss:0.72244
[252]	validation_0-mlogloss:0.721986
[253]

[449]	validation_0-mlogloss:0.650455
[450]	validation_0-mlogloss:0.65012
[451]	validation_0-mlogloss:0.649797
[452]	validation_0-mlogloss:0.649519
[453]	validation_0-mlogloss:0.649237
[454]	validation_0-mlogloss:0.64904
[455]	validation_0-mlogloss:0.648765
[456]	validation_0-mlogloss:0.648511
[457]	validation_0-mlogloss:0.648096
[458]	validation_0-mlogloss:0.647909
[459]	validation_0-mlogloss:0.647752
[460]	validation_0-mlogloss:0.647516
[461]	validation_0-mlogloss:0.647289
[462]	validation_0-mlogloss:0.647022
[463]	validation_0-mlogloss:0.646731
[464]	validation_0-mlogloss:0.646479
[465]	validation_0-mlogloss:0.646136
[466]	validation_0-mlogloss:0.645956
[467]	validation_0-mlogloss:0.645679
[468]	validation_0-mlogloss:0.645434
[469]	validation_0-mlogloss:0.645136
[470]	validation_0-mlogloss:0.644935
[471]	validation_0-mlogloss:0.644756
[472]	validation_0-mlogloss:0.644494
[473]	validation_0-mlogloss:0.644262
[474]	validation_0-mlogloss:0.643936
[475]	validation_0-mlogloss:0.643818
[47

In [55]:
print("Log-loss: {0:.3f}".format(log_loss(true_label, preds_proba)))
print("Current Accuracy: {0:.3f}".format(accuracy_score(preds,true_label)))

Log-loss: 0.527
Current Accuracy: 0.787


## Generating submissions

In [151]:
my_sub = pd.DataFrame(columns={'id', 'EAP','HPL', 'MWS'})
my_sub=my_sub[['id', 'EAP','HPL', 'MWS']]
my_sub

,id,EAP,HPL,MWS


In [156]:
test['text_processed'][1162]

'signor tintontintino florenc'

In [157]:
test['text'][1162]

'There was Signor Tintontintino from Florence.'

In [164]:
query = test['text_processed'][1162]
query = cnt.transform([query])
print(query)
preds_proba = clf.predict_proba(query.tocsc())
preds_proba

  (0, 2657)	1
  (0, 4306)	1
  (0, 14368)	1


array([[ 0.28025711,  0.30506986,  0.41467297]], dtype=float32)

In [128]:
print(len(test))

8392


In [132]:
my_sub

,id,EAP,HPL,MWS


In [147]:
for i in range(len(test)):
    query = test['text'][i]
    query = cnt.transform([query])
    preds_proba = clf.predict_proba(query.tocsc())
    #print(preds_proba)
    #print("  primeira {}".format(preds_proba[0][0]))
    #print("  segunda {}".format(preds_proba[0][1]))
    #print("  terceira {}".format(preds_proba[0][2]))
    print(i)
    my_sub.loc[i] = [test['id'][i], preds_proba[0][0], preds_proba[0][1], preds_proba[0][2]]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211


IndexError: index 0 is out of bounds for axis 0 with size 0

In [121]:
my_sub

,id,EAP,HPL,MWS
0,id02310,0.049027,0.029398,0.921575
1,id24541,0.122867,0.290986,0.586147
2,id00134,0.193840,0.803738,0.002422
3,id27757,0.469151,0.496369,0.034481
4,id04081,0.911981,0.023182,0.064837
5,id27337,0.547358,0.450992,0.001650
6,id24265,0.467389,0.526977,0.005634
7,id25917,0.171242,0.386933,0.441824
8,id04951,0.968744,0.028428,0.002828
9,id14549,0.876231,0.010769,0.113001


In [122]:
my_sub.to_csv('roberto_new.csv',index=False)